In [1]:
!pip install torch torchvision torchaudio
!pip install torch_geometric
!pip install pycfg
!pip install graphviz
!apt-get install -y graphviz libgraphviz-dev
!pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycfg: filename=pycfg-0.1-py3-none-any.whl size=7152 sha256=e263e5000b9cc79487c988dcf436ce905982e29adfcf4df024fff6a79650fa38
  Stored in directory: /root/.cache/pip/wheels/5d/fc/0c/53bd7c32449a38dfeea12bd0e9d010b31f18488272f3891925
Successfully built pycfg
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk librsvg2-common l

In [ ]:
import os
import ast
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from collections import deque, defaultdict
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.distributions import Categorical
from tqdm.notebook import tqdm
import time


# =================================================================
# ControlFlowVisitor sınıfı
# =================================================================
class ControlFlowVisitor(ast.NodeVisitor):
    def __init__(self):
        self.nodes = []
        self.edges = []
        self.node_map = {}

    def _add_node(self, node, label=None):
        if node in self.node_map:
            return self.node_map[node]
        node_id = len(self.nodes)
        if label is None:
            label = type(node).__name__ if node else "Unnamed"
        self.nodes.append({'id': node_id, 'label': label})
        if node:
            self.node_map[node] = node_id
        return node_id

    def _add_edge(self, u_ids, v_id):
        if u_ids is None or v_id is None:
            return
        if not isinstance(u_ids, list):
            u_ids = [u_ids]
        for u_id in u_ids:
            if u_id is not None:
                self.edges.append((u_id, v_id))

    def build_cfg(self, source_code):
        try:
            tree = ast.parse(source_code)
        except (SyntaxError, ValueError):
            return [], []

        start_node_id = self._add_node(None, "START")
        self._visit_body(tree.body, start_node_id)
        return self.nodes, self.edges

    def _visit_body(self, body, parent_id):
        current_exit = parent_id
        for stmt in body:
            stmt_entry, stmt_exits = self.visit(stmt)
            self._add_edge(current_exit, stmt_entry)
            current_exit = stmt_exits
        return current_exit

    def visit(self, node):
        # Ziyaretçi metodunu (örn. visit_If) çağır
        method = 'visit_' + node.__class__.__name__
        visitor = getattr(self, method, self.generic_visit)
        return visitor(node)

    def generic_visit(self, node):
        node_id = self._add_node(node)
        return node_id, node_id # Giriş ve çıkış aynı

    def visit_FunctionDef(self, node):
        func_id = self._add_node(node, f"Function: {node.name}")
        # Fonksiyon içindeki akışı kendi içinde çöz, şimdilik dışarıya sadece tanımı bağla
        self._visit_body(node.body, func_id)
        return func_id, func_id

    def visit_ClassDef(self, node):
        class_id = self._add_node(node, f"Class: {node.name}")
        self._visit_body(node.body, class_id)
        return class_id, class_id

    def visit_If(self, node):
        if_id = self._add_node(node)

        # True dalının çıkışlarını bul
        true_exits = self._visit_body(node.body, if_id)

        all_exits = []
        if true_exits is not None:
             all_exits.extend(true_exits if isinstance(true_exits, list) else [true_exits])

        # False dalının çıkışlarını bul
        if node.orelse:
            else_exits = self._visit_body(node.orelse, if_id)
            if else_exits is not None:
                all_exits.extend(else_exits if isinstance(else_exits, list) else [else_exits])
        else:
            # Else yoksa, if'in kendisi de bir çıkış noktasıdır (koşul False ise)
            all_exits.append(if_id)

        return if_id, all_exits

    def visit_For(self, node):
        loop_id = self._add_node(node)
        # Döngü içindeki akışı döngü başına bağla
        body_exits = self._visit_body(node.body, loop_id)
        self._add_edge(body_exits, loop_id)
        # Döngüden çıkış (döngüye hiç girmeme veya bitme durumu)
        # Giriş: loop_id, Çıkış: loop_id (bir sonraki ifadeye bağlanmak için)
        return loop_id, loop_id

    def visit_While(self, node):
        return self.visit_For(node)


# GrafOrtami sınıfı
class GrafOrtami:
    def __init__(self, source_code, file_path, max_nodes):
        self.file_path = file_path
        self.max_nodes = max_nodes
        visitor = ControlFlowVisitor()
        nodes, edges = visitor.build_cfg(source_code)

        if not nodes: raise ValueError(f"Graf boş: {file_path}")
        self.nodes = [node['label'] for node in nodes]

        self.successors = defaultdict(list)
        for start, end in edges:
            if end not in self.successors[start]: self.successors[start].append(end)

        self.entry_node_idx = 0
        self.durum_boyutu = len(self.nodes)
        self.eylem_sayisi = max((len(succ) for succ in self.successors.values()), default=1) if self.successors else 1

        self.graph_data = self._build_graph_data(edges)
        self.reset()

    def _build_graph_data(self, edges):
        ident_matrix = torch.eye(self.durum_boyutu)
        x = torch.zeros((self.durum_boyutu, self.max_nodes))
        x[:, :self.durum_boyutu] = ident_matrix

        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous() if edges else torch.empty((2, 0), dtype=torch.long)
        return Data(x=x, edge_index=edge_index)

    def reset(self):
        self.current_node_idx = self.entry_node_idx
        self.visited_nodes = {self.current_node_idx}
        return self._get_state()

    def _get_state(self):
        # Bu sayede model her zaman aynı boyutta girdi alır.
        visited_vector = np.zeros(self.max_nodes)
        for visited_idx in self.visited_nodes:
            if visited_idx < self.max_nodes:
                visited_vector[visited_idx] = 1.0
        return (self.current_node_idx, visited_vector)

    def step(self, action_index):
        possible_next_nodes = self.successors.get(self.current_node_idx, [])
        if action_index >= len(possible_next_nodes):
            return self._get_state(), -10, False
        self.current_node_idx = possible_next_nodes[action_index]
        reward = -0.1
        if self.current_node_idx not in self.visited_nodes:
            reward = 20
            self.visited_nodes.add(self.current_node_idx)
        done = len(self.visited_nodes) == len(self.nodes)
        if done:
            reward += 200
        return self._get_state(), reward, done

class ActorCriticGNN(nn.Module):
    def __init__(self, max_nodes, max_actions):
        super(ActorCriticGNN, self).__init__()
        self.conv1 = GCNConv(max_nodes, 128)
        self.conv2 = GCNConv(128, 128)
        self.embedding_layer = nn.Sequential(
            nn.Linear(128 + max_nodes, 64),
            nn.ReLU()
        )
        self.actor_head = nn.Linear(64, max_actions)
        self.critic_head = nn.Linear(64, 1)

        self.embedding_layer = nn.Sequential(
            nn.Linear(128 + max_nodes, 64), # 128 GCN çıktısı, max_nodes padding'li visited_vector
            nn.ReLU()
        )
        self.actor_head = nn.Linear(64, max_actions)
        self.critic_head = nn.Linear(64, 1)

    def forward(self, graph_data, current_node_idx, visited_vector, action_mask=None):
        x, edge_index = graph_data.x.to(visited_vector.device), graph_data.edge_index.to(visited_vector.device)
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))

        node_embedding = x[current_node_idx]
        if visited_vector.dim() == 1: visited_vector = visited_vector.unsqueeze(0)

        combined_info = torch.cat([node_embedding, visited_vector], dim=1)
        shared_embedding = self.embedding_layer(combined_info)

        action_logits = self.actor_head(shared_embedding)

        if action_mask is not None:
            action_logits[action_mask == 0] = -1e9

        action_probs = F.softmax(action_logits, dim=-1)
        state_value = self.critic_head(shared_embedding)

        return action_probs, state_value

# PPOAgent sınıfı
class PPOAgent:
    def __init__(self, max_nodes, max_actions, device, lr=5e-5, gamma=0.99, clip_epsilon=0.1, gae_lambda=0.95, epochs=15):
        self.device = device
        self.gamma, self.clip_epsilon, self.gae_lambda, self.epochs = gamma, clip_epsilon, gae_lambda, epochs
        self.max_actions = max_actions

        self.model = ActorCriticGNN(max_nodes, max_actions).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.memory = []

    def deneyim_kaydet(self, durum, eylem, odul, log_prob, deger, done, file_path):
        self.memory.append((durum, eylem, odul, log_prob, deger, done, file_path))

    def hafizayi_temizle(self): self.memory = []

    def eylem_sec(self, durum, ortam):
        with torch.no_grad():
            durum_idx, visited_vector = durum
            visited_tensor = torch.FloatTensor(visited_vector).unsqueeze(0).to(self.device)

            possible_actions_count = len(ortam.successors.get(durum_idx, []))
            action_mask = torch.zeros((1, self.max_actions), device=self.device)
            if possible_actions_count > 0:
                action_mask[0, :possible_actions_count] = 1

            eylem_olasiliklari, durum_degeri = self.model(ortam.graph_data, [durum_idx], visited_tensor, action_mask)

            if torch.sum(eylem_olasiliklari) > 0:
                dist = Categorical(eylem_olasiliklari)
                eylem = dist.sample()
                eylem_log_prob = dist.log_prob(eylem)
            else:
                eylem = torch.tensor(random.randrange(possible_actions_count) if possible_actions_count > 0 else 0, device=self.device)
                eylem_log_prob = torch.log(torch.tensor(1.0/possible_actions_count)) if possible_actions_count > 0 else torch.tensor(0.0)

        return eylem.item(), eylem_log_prob, durum_degeri

    def ogren(self, ortam_cache):
        # 1. Belleği dosya yoluna göre grupla
        grouped_memory = defaultdict(list)
        for experience in self.memory:
            file_path = experience[-1]
            grouped_memory[file_path].append(experience)

        self.optimizer.zero_grad()

        # 2. Her dosya grubu için ayrı ayrı kayıp hesapla ve biriktir
        for file_path, group in grouped_memory.items():
            ortam = ortam_cache[file_path]
            durumlar, eylemler, oduller, eski_log_problar, degerler, doneler, _ = zip(*group)

            avantajlar, getiriler = [], []
            son_deger = degerler[-1].item() if not doneler[-1] else 0
            gae = 0
            for i in reversed(range(len(oduller))):
                delta = oduller[i] + self.gamma * son_deger * (1 - doneler[i]) - degerler[i].item()
                gae = delta + self.gamma * self.gae_lambda * (1 - doneler[i]) * gae
                getiriler.insert(0, gae + degerler[i].item())
                son_deger = degerler[i].item()

            getiriler = torch.tensor(getiriler, dtype=torch.float32).to(self.device)
            avantajlar = torch.tensor(getiriler, dtype=torch.float32).to(self.device) - torch.stack(degerler).squeeze().detach()

            eski_log_problar = torch.stack(eski_log_problar).squeeze().to(self.device).detach()
            durum_indeksleri = torch.LongTensor([d[0][0] for d in group]).to(self.device)
            durum_vektorleri = torch.FloatTensor(np.array([d[0][1] for d in group])).to(self.device)
            eylemler = torch.LongTensor([d[1] for d in group]).to(self.device)

            for _ in range(self.epochs):
                yeni_eylem_olasiliklari, yeni_degerler = self.model(ortam.graph_data, durum_indeksleri, durum_vektorleri)
                dist = Categorical(yeni_eylem_olasiliklari)
                yeni_log_problar = dist.log_prob(eylemler)
                entropy = dist.entropy().mean()

                oran = torch.exp(yeni_log_problar - eski_log_problar)
                surr1 = oran * avantajlar
                surr2 = torch.clamp(oran, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * avantajlar

                actor_loss = -torch.min(surr1, surr2).mean()
                critic_loss = F.mse_loss(yeni_degerler.squeeze(), getiriler)

                loss = actor_loss + 0.5 * critic_loss - 0.05 * entropy
                # Gradyanları biriktir
                loss.backward()

        # 3. Biriktirilen tüm gradyanlar için optimizerı bir kez çalıştır
        self.optimizer.step()
        self.hafizayi_temizle()


print("Genelleştirilmiş Ajan ve Ortam sınıfları başarıyla tanımlandı.")

Genelleştirilmiş Ajan ve Ortam sınıfları başarıyla tanımlandı.


In [ ]:
def ana_egitim_dongusu_genel():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Kullanılacak Cihaz: {device}")

    # --- AYARLAR ---
    PROJE_KLASORU = "/content/drive/MyDrive/RL_Test"
    DATASET_YOLU = os.path.join(PROJE_KLASORU, "dataset")

    # =================================================================
    # ADIM 1: VERİ SETİNİ TARA VE MAKSİMUM BOYUTLARI BUL
    # =================================================================
    print("Veri seti taranıyor ve maksimum boyutlar hesaplanıyor...")
    all_files = [os.path.join(DATASET_YOLU, f) for f in os.listdir(DATASET_YOLU) if f.endswith('.py')]
    max_nodes = 0
    max_actions = 0

    for file_path in tqdm(all_files, desc="Dosyalar Taranıyor"):
        with open(file_path, "r", encoding="utf-8") as f:
            source_code = f.read()
        visitor = ControlFlowVisitor()
        nodes, _ = visitor.build_cfg(source_code)
        if not nodes: continue

        # Geçici ortam ile boyutları al
        temp_ortam = GrafOrtami(source_code, file_path, len(nodes))
        max_nodes = max(max_nodes, temp_ortam.durum_boyutu)
        max_actions = max(max_actions, temp_ortam.eylem_sayisi)

    print(f"Tarama tamamlandı. Max Düğüm: {max_nodes}, Max Eylem: {max_actions}")

    # =================================================================
    # ADIM 2: TÜM ORTAMLARI OLUŞTUR VE CACHE'LE
    # =================================================================
    print("Tüm ortamlar oluşturuluyor ve hafızaya alınıyor (caching)...")
    ortam_cache = {}
    for file_path in tqdm(all_files, desc="Ortamlar Oluşturuluyor"):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                source_code = f.read()
            ortam_cache[file_path] = GrafOrtami(source_code, file_path, max_nodes)
        except Exception as e:
            print(f"'{file_path}' için ortam oluşturulamadı: {e}")

    gecerli_dosyalar = list(ortam_cache.keys())
    print(f"{len(gecerli_dosyalar)} adet geçerli ortam oluşturuldu.")

    # =================================================================
    # ADIM 3: AJANI VE EĞİTİM DÖNGÜSÜNÜ BAŞLAT
    # =================================================================
    ajan = PPOAgent(max_nodes, max_actions, device=device, lr=5e-5, clip_epsilon=0.1, epochs=15)

    print("\n" + "="*50 + "\nGENELLEŞTİRİLMİŞ AJAN EĞİTİMİ BAŞLIYOR\n" + "="*50)

    toplam_adim_sayisi = 5_000_000 # adım
    update_timestep = 4096
    bolum_sayaci = 0
    mevcut_adim = 0

    pbar = tqdm(total=toplam_adim_sayisi)
    while mevcut_adim < toplam_adim_sayisi:
        bolum_sayaci += 1

        # Her bölümde rastgele bir ortam seç
        secilen_dosya = random.choice(gecerli_dosyalar)
        ortam = ortam_cache[secilen_dosya]

        durum = ortam.reset()
        bolum_odulu = 0

        # update_timestep dolana kadar veya bölüm bitene kadar adım at
        for adim in range(ortam.durum_boyutu * 20):
            eylem, log_prob, deger = ajan.eylem_sec(durum, ortam)
            yeni_durum, odul, done = ortam.step(eylem)

            ajan.deneyim_kaydet(durum, eylem, odul, log_prob, deger, done, secilen_dosya)

            durum = yeni_durum
            bolum_odulu += odul
            mevcut_adim += 1
            pbar.update(1)

            if len(ajan.memory) >= update_timestep:
                ajan.ogren(ortam_cache)

            if done or mevcut_adim >= toplam_adim_sayisi:
                break

        if bolum_sayaci % 100 == 0:
            pbar.set_description(f"Bölüm {bolum_sayaci} | Son Dosya: {os.path.basename(secilen_dosya)} | Ödül: {bolum_odulu:.2f}")

    pbar.close()
    print("\nEĞİTİM TAMAMLANDI!")

    model_kayit_yolu = os.path.join(PROJE_KLASORU, "genel_kasif_ajan.pth")
    torch.save(ajan.model.state_dict(), model_kayit_yolu)
    print(f"Genel kaşif modeli başarıyla '{model_kayit_yolu}' adresine kaydedildi.")

# Eğitimi başlat
ana_egitim_dongusu_genel()

Kullanılacak Cihaz: cuda
Veri seti taranıyor ve maksimum boyutlar hesaplanıyor...


Dosyalar Taranıyor:   0%|          | 0/66 [00:00<?, ?it/s]

Tarama tamamlandı. Max Düğüm: 55, Max Eylem: 2
Tüm ortamlar oluşturuluyor ve hafızaya alınıyor (caching)...


Ortamlar Oluşturuluyor:   0%|          | 0/66 [00:00<?, ?it/s]

66 adet geçerli ortam oluşturuldu.

GENELLEŞTİRİLMİŞ AJAN EĞİTİMİ BAŞLIYOR


  0%|          | 0/5000000 [00:00<?, ?it/s]

/tmp/ipython-input-2433602042.py:288: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  avantajlar = torch.tensor(getiriler, dtype=torch.float32).to(self.device) - torch.stack(degerler).squeeze().detach()
/tmp/ipython-input-2433602042.py:306: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(yeni_degerler.squeeze(), getiriler)



EĞİTİM TAMAMLANDI!
Genel kaşif modeli başarıyla '/content/drive/MyDrive/RL_Test/genel_kasif_ajan.pth' adresine kaydedildi.


In [ ]:
import os
import torch
from tqdm.notebook import tqdm
import numpy as np

def test_genel_ajan():
    """
    Eğitilmiş genel kaşif modelini yükler ve veri setindeki tüm dosyalar
    üzerinde performansını test eder.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Test için kullanılacak cihaz: {device}")

    # --- AYARLAR ---
    PROJE_KLASORU = "/content/drive/MyDrive/RL_Test"
    DATASET_YOLU = os.path.join(PROJE_KLASORU, "dataset")
    MODEL_YOLU = os.path.join(PROJE_KLASORU, "genel_kasif_ajan.pth")

    if not os.path.exists(MODEL_YOLU):
        print(f"HATA: Model dosyası bulunamadı: {MODEL_YOLU}")
        return

    # 1. Ortamları ve max boyutları yeniden hazırla (hücrenin tek başına çalışabilmesi için)
    print("Test için ortamlar hazırlanıyor...")
    all_files = [os.path.join(DATASET_YOLU, f) for f in os.listdir(DATASET_YOLU) if f.endswith('.py')]
    max_nodes, max_actions = 0, 0
    for file_path in all_files:
        with open(file_path, "r", encoding="utf-8") as f: source_code = f.read()
        visitor = ControlFlowVisitor(); nodes, _ = visitor.build_cfg(source_code)
        if not nodes: continue
        temp_ortam = GrafOrtami(source_code, file_path, len(nodes))
        max_nodes = max(max_nodes, temp_ortam.durum_boyutu)
        max_actions = max(max_actions, temp_ortam.eylem_sayisi)

    ortam_cache = {}
    for file_path in all_files:
        try:
            with open(file_path, "r", encoding="utf-8") as f: source_code = f.read()
            ortam_cache[file_path] = GrafOrtami(source_code, file_path, max_nodes)
        except Exception: continue

    print(f"Test için {len(ortam_cache)} ortam hazır. Max Düğüm: {max_nodes}, Max Eylem: {max_actions}")

    # --- EĞİTİLMİŞ AJANI YÜKLE ---
    print(f"Eğitilmiş model yükleniyor: {MODEL_YOLU}")
    test_ajani = PPOAgent(max_nodes, max_actions, device)
    test_ajani.model.load_state_dict(torch.load(MODEL_YOLU, map_location=device))
    test_ajani.model.eval()  # Modeli Değerlendirme (Evaluation) moduna al
    print("Model başarıyla yüklendi ve test moduna alındı.")

    # --- TEST DÖNGÜSÜNÜ BAŞLAT ---
    results = []
    for file_path, ortam in tqdm(ortam_cache.items(), desc="Dosyalar Test Ediliyor"):
        durum = ortam.reset()
        adim_sayisi = 0
        bolum_odulu = 0
        max_test_adimi = ortam.durum_boyutu * 5

        for _ in range(max_test_adimi):
            with torch.no_grad():
                # AJANIN KARAR MEKANİZMASI (DETERMİNİSTİK)
                durum_idx, visited_vector = durum
                visited_tensor = torch.FloatTensor(visited_vector).unsqueeze(0).to(device)

                possible_actions_count = len(ortam.successors.get(durum_idx, []))
                action_mask = torch.zeros((1, test_ajani.max_actions), device=device)
                if possible_actions_count > 0:
                    action_mask[0, :possible_actions_count] = 1

                eylem_olasiliklari, _ = test_ajani.model(ortam.graph_data, [durum_idx], visited_tensor, action_mask)
                eylem = torch.argmax(eylem_olasiliklari).item()

            yeni_durum, odul, done = ortam.step(eylem)

            durum = yeni_durum
            bolum_odulu += odul
            adim_sayisi += 1

            if done:
                break

        # Sonuçları kaydet
        kesfedilen_düğüm = len(ortam.visited_nodes)
        toplam_düğüm = ortam.durum_boyutu
        basari = kesfedilen_düğüm == toplam_düğüm
        results.append({
            "dosya": os.path.basename(file_path),
            "basari": basari,
            "kesif_orani": f"{kesfedilen_düğüm}/{toplam_düğüm}",
            "adim_sayisi": adim_sayisi,
            "odul": bolum_odulu
        })

    # --- SONUÇLARI GÖSTER ---
    print("\n" + "="*50 + "\nTEST SONUÇLARI\n" + "="*50)
    basarili_sayisi = 0
    basarili_adimlar = []
    basarili_oduller = []

    for res in sorted(results, key=lambda x: x['basari'], reverse=True):
        durum_isareti = "✓ BAŞARILI" if res["basari"] else "✗ BAŞARISIZ"
        print(f"{durum_isareti:<12} | Dosya: {res['dosya']:<40} | Keşif: {res['kesif_orani']:<10} | Adım: {res['adim_sayisi']:<5} | Ödül: {res['odul']:.2f}")
        if res["basari"]:
            basarili_sayisi += 1
            basarili_adimlar.append(res["adim_sayisi"])
            basarili_oduller.append(res["odul"])

    print("\n" + "="*50 + "\nGENEL ÖZET\n" + "="*50)
    toplam_dosya = len(results)
    basari_orani = (basarili_sayisi / toplam_dosya) * 100 if toplam_dosya > 0 else 0
    print(f"Toplam Test Edilen Dosya: {toplam_dosya}")
    print(f"Tamamen Keşfedilen Dosya Sayısı: {basarili_sayisi}")
    print(f"Genel Başarı Oranı: {basari_orani:.2f}%")

    if basarili_sayisi > 0:
        ortalama_adim = np.mean(basarili_adimlar)
        ortalama_odul = np.mean(basarili_oduller)
        print(f"\nBaşarılı dosyalarda ortalama adım sayısı: {ortalama_adim:.2f}")
        print(f"Başarılı dosyalarda ortalama ödül: {ortalama_odul:.2f}")


# Testi başlat
test_genel_ajan()

Test için kullanılacak cihaz: cuda
Test için ortamlar hazırlanıyor...
Test için 66 ortam hazır. Max Düğüm: 55, Max Eylem: 2
Eğitilmiş model yükleniyor: /content/drive/MyDrive/RL_Test/genel_kasif_ajan.pth
Model başarıyla yüklendi ve test moduna alındı.


Dosyalar Test Ediliyor:   0%|          | 0/66 [00:00<?, ?it/s]


TEST SONUÇLARI
✓ BAŞARILI   | Dosya: factorial_analysis_no_visuals.py         | Keşif: 6/6        | Adım: 5     | Ödül: 300.00
✓ BAŞARILI   | Dosya: find_divisors_analysis_no_visuals.py     | Keşif: 6/6        | Adım: 5     | Ödül: 300.00
✓ BAŞARILI   | Dosya: find_number_analysis_no_visuals.py       | Keşif: 12/12      | Adım: 11    | Ödül: 420.00
✓ BAŞARILI   | Dosya: hello_analysis_no_visuals.py             | Keşif: 4/4        | Adım: 3     | Ödül: 260.00
✓ BAŞARILI   | Dosya: mean_analysis_no_visuals.py              | Keşif: 7/7        | Adım: 6     | Ödül: 320.00
✓ BAŞARILI   | Dosya: odd_even_analysis_no_visuals.py          | Keşif: 6/6        | Adım: 5     | Ödül: 300.00
✓ BAŞARILI   | Dosya: print_screen_numbers_analysis_no_visuals.py | Keşif: 12/12      | Adım: 11    | Ödül: 420.00
✓ BAŞARILI   | Dosya: print_simple_number_analysis_no_visuals.py | Keşif: 6/6        | Adım: 5     | Ödül: 300.00
✓ BAŞARILI   | Dosya: reverse_string_analysis_no_visuals.py    | Keşif: 7/7        